# Part 1

 Create a new Jupyter notebook called 'SEDS-HW3.ipynb' and finish the in class excersize of creating a functionalized block of code to compute the pairwise correlation between rows in a pandas data frame.  Use Markdown between cells to document your work.  Be sure to use docstrings for all of your functions.  _Hint / requirement_ You should have three functions.  One for the whole entity `pairwise_correlation`, one for comparing two rows `corr_rowi_rowj`, and one for comparing one row to all the other rows `corr_rowi_vs_all`.

#### Here is the pseudo code to formulate the idea. 

def pairwise_correlation:

create empty correlation_matrix
for row_i in dataframe
    for row_i compare to all rows in dataframe
        compute correlation of row_i to row_j
        save correlation to correlation_matrix
return correlation_matrix
return

#### Imports are listed here

In [1]:
import pandas as pd
import numpy as np

#### First we will create the sample dataframe

In [25]:
df = pd.DataFrame([[-1, 0, 1], [1, 0, -1], [.5, 0, .5]], index = ['A', 'B', 'C'], columns = ['v1', 'v2', 'v3'])

df

,v1,v2,v3
A,-1.0,0,1.0
B,1.0,0,-1.0
C,0.5,0,0.5


#### We need to create three seperate functions that will all work together to solve the idea outlined in the pseudo code. The first function will serve the purpose of comparing one row i, to another row, j, and computing the correlation of those two. 

In [46]:
#compute correlation of row_i to row_j

def corr_row_i_row_j(row_i, row_j):
    """This function will compute the correlation between one row, i, and a second row, j"""
    return row_i.corr(row_j)

In [47]:
#quick test, the correlation of row A to B should be -1

corr_row_i_row_j(df.loc['A'], df.loc['B'])

-1.0

#### This next function will then include the function above to compare the row, i, to every row in the dataframe and compute the correlation for each one. While we hav'nt creted it yet, this fucntion will also save the correlations to our correlation matrix, labeled as corr_matrix

In [85]:
#for row_i compare to all rows in dataframe

def corr_row_i_vs_all(df, index_i, row_i, corr_matrix):
    """This function, after having specified a row, i, will compute the correlation matrix between row i
    and every other row in the dataframe"""
    for index_j, row_j in df.iterrows():# pd.itterows will itterate over every row in the dataframe
        corr_matrix.loc[index_i, index_j] = corr_row_i_row_j(row_i, row_j) #computes correlation and saves to matrix
        corr_matrix.loc[index_j, index_i] = corr_matrix.loc[index_i, index_j]
    return



#### Now we will create the main function, which creates the emtpy correlation matrix, calls every row in the dataframe, and uses the two previous functions to compute the correlation matrix between every other row. Finally, it will return the filled correlation matrix.

In [86]:
def pairwise_correlation(df):
    """This function will create an empty correlation matrix and itterate over every row to compute the correlation 
    against every row, and return the filled correlation matrix with the results."""
    corr_matrix = pd.DataFrame()
    for index_i, row_i in df.iterrows():
        corr_row_i_vs_all(df, index_i, row_i, corr_matrix)
    return corr_matrix

#### Now this should give us the entire correlation matrix for the dataframe

In [87]:
pairwise_correlation(df)

,A,B,C
A,1.0,-1.0,0.0
B,-1.0,1.0,0.0
C,0.0,0.0,1.0


# Part 2:

The three fucntions were moved to df_utils.py, and imported into this notebook below.

In [91]:
import df_utils

# Part 3 and 4

For each function, create cell with _assert_ statements that tests various aspects of your function, including handling oddball inputs, expected outputs (size, shape and values).  Try to have at least three tests per function.

Convert the cells to unit test functions and save them to `test_df_utils.py`.  _Hint_ Don't forget to name the functions with the ``test_`` prefix!

Similar to the example in the notebook, we will first create a test function for corr_row_i_row_j. As noted, it is probably not very useful to have a test for this function, as it only calls on another well documented and assumingly well maintained pandas function. Nevertheless, it is a good intro excercise and leads into tackling unit tests for the other more compelx functions.

In [96]:
def test_corr_row_i_row_j():
    """This is a test function for corr_row_i_row_j"""
    #introducing the dataframe 
    df = pd.DataFrame([[-1, 0, 1], [1, 0, -1], [.5, 0, .5]], index = ['A', 'B', 'C'], columns = ['v1', 'v2', 'v3'])
    #assert statements
    assert int(df_utils.corr_row_i_row_j(df.loc['A'], df.loc['A'])) == 1 #value of correlation for row A to A should be 1
    assert int(df_utils.corr_row_i_row_j(df.loc['A'], df.loc['B'])) == -1 #value of correlation for row A to B should be -1
    assert int(df_utils.corr_row_i_row_j(df.loc['A'], df.loc['C'])) == 0 #value of correltion for row A to B should be 0
    return

Knowing what the value of the correlation between row A with the other rows, we asserted that the correlation function should return those values

In [97]:
test_corr_row_i_row_j()

Now we will create a unit test for the second function, corr_row_i_vs_all

In [168]:
def test_corr_row_i_vs_all():
    """This is a test function for corr_row_i_vs_all"""
    #introducing the dataframe 
    df = pd.DataFrame([[-1, 0, 1], [1, 0, -1], [.5, 0, .5]], index = ['A', 'B', 'C'], columns = ['v1', 'v2', 'v3'])
    
    #introducing empty correlation matrix
    corr_matrix = pd.DataFrame()
    
    #Checking that dataframe has more than one row for pd.iterrows to function properly
    assert len(df.index) > int(1), "pd.iterrows cannot iterate because dataframe only has one row" 
    
    #Checking that all rows in correlation matrix get filled, without any NaN's.
    df_utils.corr_row_i_vs_all(df, 'A', df.loc['A'], corr_matrix)
    df_utils.corr_row_i_vs_all(df, 'B', df.loc['B'], corr_matrix)
    df_utils.corr_row_i_vs_all(df, 'C', df.loc['C'], corr_matrix)
    
    assert df.isnull().values.any() == False,  "null (NaN) values are present, not iterating correctly"
    
   #Checking that the function is outputting the correct matrix symmetry
    assert corr_matrix.at['A', 'B'] == corr_matrix.at['B', 'A'], "The function is not outputting the correct symmetry"
    assert corr_matrix.at['B', 'C'] == corr_matrix.at['C', 'B'], "The function is not outputting the correct symmetry"
    
    return

Here, we asserted that (assuming the dataframe passed is indeed a dataframe and isnt empty, which will be tested in the next function) the dataframe should have more than one row, because you can't really iterate over rows if you only have one row to iterate to. Of course you could still get the correlation of a single row to itself, but this will always be 1 and is trivial. Also, we asserted that all rows in the dataframe should be filled, but without any NaN's. I was actually having this issue while writing the original function, where the code would run but some rows had NaN's, so I thoguht this would be useful. Finally we asserted that row A column B in the correlation matrix should be the same as row B column A, and so forth. 

In [169]:
test_corr_row_i_vs_all()

The final test function will be for pairwise_correlation

In [172]:
def test_pairwise_correlation():
    """This is a test function for pairwise_correlation"""

    #introducing the dataframe 
    df = pd.DataFrame([[-1, 0, 1], [1, 0, -1], [.5, 0, .5]], index = ['A', 'B', 'C'], columns = ['v1', 'v2', 'v3'])
    
    #introducing empty correlation matrix
    corr_matrix = pd.DataFrame()
    
    #Checking that the dataframe and correaltion matrix passed are indeed dataframes, and not an integer, string, etc. 
    assert isinstance(df, pd.DataFrame), "Data is not part of a dataframe" 
    assert isinstance(corr_matrix, pd.DataFrame), "correlation matrix is not part of a dataframe"
    
    #Checking that the datarame passed is not empty, for example, if one mixed up labeling of the dataframe and corr_matrix
    assert df.empty == False, "Dataframe is empty" 
    
    #Checking that the correlation matrix is empty, so the correlations computed by the fucntion can be saved in the matrix.
    assert corr_matrix.empty == True, "Correlation matrix needs to be empty to correctly save the computations"
    
    #Checking that dataframe has more than one row for pd.iterrows to function properly
    assert len(df.index) > int(1), "pd.iterrows cannot iterate because dataframe only has one row" 
    
    return

Above, we asserted that the data and correlation matrix passed are dataframes. We asserted that the dataframe for the data should not be empty, however the correlation amtrix dataframe should be. We also once again asserted that the data in the dataframe should have multiple rows. 

In [174]:
test_pairwise_correlation()

These test functions were copied and saved in `test_df_utils.py`

# Part 5


These unit tests were ran with `nosetests` at the command line and the output pasted to the README file (Also pasted here below)

```Ran 3 tests in 0.685s```

 ```OK```